<a href="https://colab.research.google.com/github/Anyaoma/My-projects/blob/main/Algorithmic_Trading_with_Oanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import json
import math
import pandas as pd
import numpy as np

In [ ]:
import oandapyV20
import oandapyV20.endpoints
from oandapyV20 import API
import oandapyV20.endpoints.accounts as v20Accounts
import oandapyV20.endpoints.instruments as v20Instruments
import oandapyV20.endpoints.pricing as v20Pricing
import oandapyV20.endpoints.orders as v20Orders
import oandapyV20.endpoints.positions as v20Positions
import oandapyV20.contrib.requests as v20Requests
import oandapyV20.endpoints.trades as v20Trades
from oandapyV20.contrib.factories import InstrumentsCandlesFactory

In [ ]:
class OandaTrader1:

    def __init__(self, file_name=None):
        self.credentials = self.GetCredentials(file_name)
        self.Broker = self.Login()
        self.Markets = self.GetMarkets()

        #Load credentials from file

    def GetCredentials(self, file_name):
        credentials = {}
        if os.path.exists(file_name):
            try:
                filehandle = open(file_name,'r')
                credentials['AccountID'] = filehandle.readline().strip()
                credentials['BearerToken'] = filehandle.readline().strip()
                filehandle.close()
            except:
                print(f'Please check your {file_name} missing or malformed data')
                sys.exit(1)
        else:
            print(f'you must save your OANDA account credentials to: {file_name}')
            sys.exit(1)

        return credentials

    #Login to the exchange

    def Login(self):
        try:
            oanda = API(access_token = self.credentials['BearerToken'])
        except Exception as err:
            print('Error:',str(err))
            sys.exit(1)

        return oanda

    #Handle all OANDA requests with retry loop

    def API(self, function, req):
        retry = 0
        RetryLimit = 3

        done = False
        while not done:
            try:
                result = self.Broker.request(req)
            except Exception as err:
                if retry > RetryLimit:
                    print(str(err))
                    sys.exit(1)
                else:
                    em = str(err)
                    print(f"{function}:Retrying({retry+1}), {em}")
            else:
                done = True
            retry+=1
        return result

    #Retrieve tradeable instruments

    def GetMarkets(self):
        req = v20Accounts.AccountInstruments(accountID=self.credentials['AccountID'])
        results = self.API('GetMarket', req)
        return results

    #Retrive ticker information

    def GetTicker(self,symbol):
        params = {'instruments':symbol}
        req = v20Pricing.PricingInfo(accountID=self.credentials['AccountID'],params=params)
        ticker = self.API('GetTicker', req)

        #Build the forex pair dictionary
        ForexPair = {}
        ForexPair['Time'] = pd.Timestamp(ticker['time'])
        ForexPair['Ask'] = round(float(ticker['prices'][0]['asks'][0]['price']),5)
        ForexPair['Bid'] = round(float(ticker['prices'][0]['bids'][0]['price']),5)
        ForexPair['Spread'] = round(ForexPair['Ask']-ForexPair['Bid'],5)
        return ForexPair

     # Read the order book

    def GetOrderBook(self, symbol):
        req = v20Instruments.InstrumentsOrderBook(instrument=symbol,params={})
        results = self.API("GetOrderBook",req)
        return results['orderBook']['buckets']

     # Read the currnt open positions
    def GetOpenPositions(self, symbol):
        symbol = symbol.replace('/','_')
        params={"instruments":symbol }
        req=v20Trades.TradesList(self.credentials['AccountID'],params=params)
        results=self.API("GetOpenPosiions",req)
        return results['trades']

    # Place a trade/open a position

    def PlaceOrder(self, pair, orderType, units, price=None):
        pair = pair.replace('/','_')
        orderType = orderType.upper()
        order = {}
        if orderType == 'LIMIT':
            order['price'] = str(price)
            order['timeInForce'] = 'GTC'
        order['units'] = str(units)
        order['instrument'] = pair
        order['type'] = orderType
        order['positionFill'] = 'DEFAULT'
        params = {}
        params['order'] = order
        res=v20Orders.OrderCreate(self.credentials['AccountID'],data=params)
        results=self.API("OrderCreate",res)
        return results

    #Close a trade or position

    def CloseOrder(self, pair, units, price=None):
        params = {}
        if str(units).upper()!='ALL':
            #amount is STR, need float for abs()
            units = float(units)
            if float(units)>=0:
                params['longUnits']=str(math.floor(abs(units)))
            else:
                params['shortUnits']=str(math.floor(abs(units)))
        else:
            params['longUnits'] = 'ALL'
        res=v20Positions.PositionClose(self.credentials['AccountID'],instrument=pair,data=params)
        results=self.API("PositionClose",res)


        return results

    #Get historial data
    def get_history(self,instrument, time_frame, from_, end ):
        instrument, granularity = instrument, time_frame
        _from = from_

        params = {"from": _from,
                  "to": end,
              "granularity": granularity,
              }

        all_candle = []
        output_directory = "/tmp"
        file_path = os.path.join(output_directory, "{}.{}".format(instrument, granularity))
        os.makedirs(output_directory, exist_ok=True)

        for req in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = self.API('GetHistory',req)
            all_candle.extend(response.get('candles'))
            #all_candles.extend(r.response.get('candles'))

        with open(file_path, "w") as OUT:
            json.dump(all_candle, OUT, indent=2)

        with open(file_path, "r") as file:
            data = json.load(file)

        candles = data if isinstance(data, list) else data['candles']
        df = pd.DataFrame(candles)

        columns = ['volume','time','open','high','low','close']
        open_price = []
        high_price = []
        low_price = []
        close_price = []
        for price in df['mid']:
            open_price.append(price['o'])
            high_price.append(price['h'])
            low_price.append(price['l'])
            close_price.append(price['c'])

        df['open'] = open_price
        df['high'] = high_price
        df['low'] = low_price
        df['close'] = close_price

        df = df[columns]

        return df

    #Get account summary
    def get_account_summary(self):
        req= v20Accounts.AccountSummary(accountID=self.credentials['AccountID'])
        request = self.API('GetAccountSummary', req)
        print(json.dumps(request, indent=2))


In [ ]:
class MomentumStrategy(OandaTrader1):
    def __init__(self,  symbol, time_lenght, mom,file_name):
        super().__init__(file_name)
        self.position = 0
        self.instrument = symbol
        self.time_lenght = time_lenght
        self.momentum = mom
        #self.units = units
        self.min_lenght = self.momentum + 1
        self.raw_data = pd.DataFrame()
        self.ordertype = 'MARKET'

    def place_trade(self,time, bid, ask,units):
        self.raw_data = pd.concat([self.raw_data,pd.DataFrame({'bid':float(bid),'ask':float(ask)}, index = [pd.Timestamp(time)])])
        self.data = self.raw_data.resample(self.time_lenght, label='right').last().ffill().iloc[:-1]
        self.data['mid'] = self.data.mean(axis=1)
        self.data['returns'] = np.log(self.data['mid']/self.data['mid'].shift())
        self.data['position'] = np.sign(self.data['returns'].rolling(self.momentum).mean())
        if len(self.data) > self.min_lenght:
            print('running-------')
            self.min_lenght += 1
            if self.data['position'].iloc[-1] == 1:
                print('runs baby')
                if self.position == 0:
                    self.PlaceOrder(self.instrument, self.ordertype, units)
                elif self.position == -1:
                    self.PlaceOrder(self.instrument, self.ordertype, units*2)
                self.position = 1
            elif self.data['position'].iloc[-1] == -1:
                if self.position == 0:
                    self.PlaceOrder(self.instrument, self.ordertype, -units)
                elif self.position == 1:
                    self.PlaceOrder(self.instrument, self.ordertype, -units*2)
        #return data

    def stream_data(self, symbol,units, stop):
        params = {"instruments": symbol }
        req = v20Pricing.PricingStream(accountID=self.credentials['AccountID'], params=params)
        results = self.API("StreamData",req)

        maxrecs = stop
        done = False
        while not done:
            try:
                for ticks in results:
                    new_data = {}
                    #data.append(ticks)
                    new_data['time'] = ticks['time']
                    new_data['bid'] = ticks['bids'][0]['price']
                    new_data['ask'] = ticks['asks'][0]['price']

                    self.place_trade(time = new_data['time'], bid = float(new_data['bid']), ask = float(new_data['ask']), units=units )

                    print(new_data['time'])
                    print(new_data['bid'])
                    print(new_data['ask'])
                    maxrecs -= 1
                    if maxrecs == 0:
                        results.terminate()
            except KeyError as err:
                del new_data['time']
                continue
            except StreamTerminated as e:
                done = True
                break
            else:
                self._place_trade(time = new_data['time'], bid = float(new_data['bid']), ask = float(new_data['ask']), units=units )


     #Get account summary
    def get_account_summary(self):
        req= v20Accounts.AccountSummary(accountID=self.credentials['AccountID'])
        request = self.API('GetAccountSummary', req)
        print(json.dumps(request, indent=2))



In [ ]:
trader = MomentumStrategy(file_name = "credentials.txt", symbol = 'EUR_USD', time_lenght = '3s',mom=6)

In [ ]:
openOrders=trader.GetOpenPositions('EUR_USD')

for o in openOrders:
    iu=int(o['currentUnits'])
    price=float(o['price'])
    if price>=0:
        side='Long'
    else:
        side='Shrt'
    print(f"{o['openTime']} {o['id']:8} {side:4} {o['state']:7} {iu:16.8f} {price:16.8f}")

2024-06-28T13:07:09.377561993Z 157      Long OPEN       -100.00000000       1.07052000


In [ ]:
result=trader.CloseOrder('EUR_USD',  units=-100, price=None)
openOrders=broker.GetOpenPositions('EUR_USD')

for o in openOrders:
    iu=int(o['currentUnits'])
    price=float(o['price'])
    if price>=0:
        side='Long'
    else:
        side='Shrt'
    print(f"{o['openTime']} {o['id']:8} {side:4} {o['state']:7} {iu:16.8f} {price:16.8f}")

In [ ]:
start = '2020-06-08T00:00:00Z'
end = '2020-06-13T00:00:00Z'

In [ ]:
broker=OandaTrader1("credentials.txt")

In [ ]:
raw = broker.get_history(instrument='EUR_USD', time_frame='M10', from_ =start , end=end )

In [ ]:
raw

,volume,time,open,high,low,close
0,393,2020-06-08T00:00:00.000000000Z,1.13158,1.13158,1.13101,1.13112
1,314,2020-06-08T00:10:00.000000000Z,1.13110,1.13133,1.13048,1.13050
2,283,2020-06-08T00:20:00.000000000Z,1.13048,1.13148,1.13047,1.13117
3,372,2020-06-08T00:30:00.000000000Z,1.13115,1.13115,1.13032,1.13032
4,342,2020-06-08T00:40:00.000000000Z,1.13031,1.13075,1.13024,1.13071
...,...,...,...,...,...,...
696,221,2020-06-12T20:10:00.000000000Z,1.12572,1.12593,1.12532,1.12568
697,163,2020-06-12T20:20:00.000000000Z,1.12569,1.12578,1.12532,1.12558
698,192,2020-06-12T20:30:00.000000000Z,1.12560,1.12573,1.12534,1.12543
699,219,2020-06-12T20:40:00.000000000Z,1.12544,1.12594,1.12528,1.12542


In [ ]:
raw.tail()

,volume,time,open,high,low,close
696,221,2020-06-12T20:10:00.000000000Z,1.12572,1.12593,1.12532,1.12568
697,163,2020-06-12T20:20:00.000000000Z,1.12569,1.12578,1.12532,1.12558
698,192,2020-06-12T20:30:00.000000000Z,1.12560,1.12573,1.12534,1.12543
699,219,2020-06-12T20:40:00.000000000Z,1.12544,1.12594,1.12528,1.12542
700,296,2020-06-12T20:50:00.000000000Z,1.12544,1.12624,1.12541,1.12554


In [ ]:
raw.to_csv('rawdata.csv', index=False)